In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import numpy as np
import cv2

def load_train_lfw_data(data_dir):
    # Danh sách tất cả các tên thư mục (mỗi thư mục chứa các ảnh của một người)
    person_dirs = glob.glob(os.path.join(data_dir, '*'))

    images = []
    names = []

    for person_dir in person_dirs:
        person_name = os.path.basename(person_dir)  # Tên người
        image_paths = glob.glob(os.path.join(person_dir, '*.jpg'))

        for img_path in image_paths:
            img = cv2.imread(img_path)  # Đọc ảnh với 3 kênh màu (RGB)
            img = cv2.resize(img, (224, 224))  # Resize về kích thước mong muốn
            images.append(img)
            names.append(person_name)

    images = np.array(images)
    names = np.array(names)

    return images, names

data_dir = '/content/drive/MyDrive/Colab Notebooks/Train_LFW'
X, names = load_train_lfw_data(data_dir)


In [ ]:
# Encode nhãn thành số nguyên
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
names_encoded = le.fit_transform(names)

# Lưu encoder để dùng sau
import joblib
joblib.dump(le, '/content/drive/MyDrive/label_encoder_lfw.pkl')

['/content/drive/MyDrive/label_encoder_lfw.pkl']

In [ ]:
# Chia train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, names_encoded, test_size=0.2, stratify=names_encoded)

# Thông tin dữ liệu
print(X_train.shape, X_test.shape)

(412, 224, 224, 3) (103, 224, 224, 3)


In [ ]:
# Tạo mô hình backbone
import tensorflow as tf
from tensorflow.keras.layers import Dense, Lambda, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

def _base_network():
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

    #Mở 4 lớp cuối
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    x = Flatten()(base_model.output)
    x = Dense(128)(x)
    norm2 = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    model = Model(inputs=base_model.input, outputs=norm2)
    return model

# Khởi tạo mô hình
model = _base_network()
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Dùng Triplet Loss
import tensorflow_addons as tfa
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tfa.losses.TripletSemiHardLoss()
)

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
gen_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(64).repeat()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Tạo callback để lưu mô hình sau mỗi epoch
checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/facenet_ver2.h5',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,  # Lưu toàn bộ mô hình (weights và architecture)
    mode='min',
    verbose=1  # In ra thông tin khi lưu checkpoint
)

In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=15,
    callbacks=[checkpoint_callback]  # Thêm callback để checkpoint
)

Epoch 1/15
30/30 [==============================] - ETA: 0s - loss: 0.8742 
Epoch 1: loss improved from inf to 0.87417, saving model to /content/drive/MyDrive/facenet_ver2.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 1301s 43s/step - loss: 0.8742
Epoch 2/15
30/30 [==============================] - ETA: 0s - loss: 0.5677 
Epoch 2: loss improved from 0.87417 to 0.56771, saving model to /content/drive/MyDrive/facenet_ver2.h5
30/30 [==============================] - 1298s 43s/step - loss: 0.5677
Epoch 3/15
30/30 [==============================] - ETA: 0s - loss: 0.3121 
Epoch 3: loss improved from 0.56771 to 0.31210, saving model to /content/drive/MyDrive/facenet_ver2.h5
30/30 [==============================] - 1295s 43s/step - loss: 0.3121
Epoch 4/15
30/30 [==============================] - ETA: 0s - loss: 0.1540 
Epoch 4: loss improved from 0.31210 to 0.15396, saving model to /content/drive/MyDrive/facenet_ver2.h5
30/30 [==============================] - 1268s 43s/step - loss: 0.1540
Epoch 5/15
30/30 [==============================] - ETA: 0s - loss: 0.0576 
Epoch 5: loss improved from 0.15396 to 0.05757, saving model to /content/drive/MyDrive/facenet_ver2.h5


KeyboardInterrupt: 

# Kiểm tra độ chính xác trên bộ dữ liệu dùng để Train

In [ ]:
# Trích xuất embedding cho tập train và test
X_train_vecs = model.predict(X_train)
X_test_vecs = model.predict(X_test)

4/4 [==============================] - 61s 14s/step


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def _most_similarity(train_vecs, test_vec, train_labels):
    sims = cosine_similarity(train_vecs, test_vec)  # shape: (n_train, 1)
    idx = np.argmax(sims)
    return train_labels[idx]

In [ ]:
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test_vecs:
    vec = vec.reshape(1, -1)
    y_pred = _most_similarity(X_train_vecs, vec, y_train)
    y_preds.append(y_pred)

print("Accuracy:", accuracy_score(y_test, y_preds))

Accuracy: 0.7961165048543689


# Kiểm tra độ chính xác trên bộ LFW

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_ver2.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import pandas as pd
import os

img_dir = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled"
pairs_file = "/content/drive/MyDrive/Colab Notebooks/lfw/pairs.csv"

df = pd.read_csv(pairs_file, usecols=['name', 'imagenum1', 'imagenum2', 'Unnamed: 3'])

positive_pairs = df.iloc[:300]
negative_pairs = df.iloc[300:600]

pairs = []

# Positive pairs
for _, row in positive_pairs.iterrows():
    name = row['name']
    img1 = f"{img_dir}/{name}/{name}_{int(row['imagenum1']):04d}.jpg"
    img2 = f"{img_dir}/{name}/{name}_{int(row['imagenum2']):04d}.jpg"
    pairs.append((img1, img2, 1))

# Negative pairs
for _, row in negative_pairs.iterrows():
    name1 = row['name']
    img1 = f"{img_dir}/{name1}/{name1}_{int(row['imagenum1']):04d}.jpg"
    name2 = row['imagenum2']
    img2 = f"{img_dir}/{name2}/{name2}_{int(row['Unnamed: 3']):04d}.jpg"
    pairs.append((img1, img2, 0))

In [ ]:
import cv2
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Mặc định là BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Đổi sang RGB
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Đánh giá mô hình trên 1 ngưỡng
def evaluate_model(model, pairs, threshold):
    correct = 0
    total = len(pairs)
    for img1_path, img2_path, label in tqdm(pairs, desc=f"Evaluating (threshold={threshold:.2f})", leave=False):
        try:
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)

            vec1 = model.predict(img1)[0]
            vec2 = model.predict(img2)[0]

            sim = cosine_similarity([vec1], [vec2])[0][0]
            pred = 1 if sim >= threshold else 0

            if pred == label:
                correct += 1
        except Exception as e:
            print(f"Lỗi xử lý cặp ảnh {img1_path} và {img2_path}: {e}")
    return correct / total if total > 0 else 0

# Tìm threshold tốt nhất
def find_best_threshold(model, pairs, thresholds):
    best_threshold = 0
    best_acc = 0
    for threshold in thresholds:
        acc = evaluate_model(model, pairs, threshold)
        print(f"Threshold: {threshold:.2f} → Accuracy: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

# Danh sách ngưỡng cần kiểm tra
thresholds_to_try = [0.73, 0.75, 0.77, 0.80, 0.85, 0.90, 0.95]

# Tìm threshold tối ưu
best_threshold, best_accuracy = find_best_threshold(model, pairs, thresholds_to_try)
print("\nBest threshold:", best_threshold)
print("Best accuracy :", best_accuracy)

Evaluating (threshold=0.73):   0%|          | 0/600 [00:00<?, ?it/s]

1/1 [==============================] - 1s 527ms/step


Evaluating (threshold=0.73):   0%|          | 1/600 [00:01<15:49,  1.58s/it]

1/1 [==============================] - 1s 514ms/step


Evaluating (threshold=0.73):   0%|          | 2/600 [00:02<13:13,  1.33s/it]

...

Evaluating (threshold=0.95): 100%|█████████▉| 598/600 [14:30<00:02,  1.48s/it]

1/1 [==============================] - 1s 575ms/step


Evaluating (threshold=0.95): 100%|█████████▉| 599/600 [14:31<00:01,  1.43s/it]

1/1 [==============================] - 1s 563ms/step


Threshold: 0.95 → Accuracy: 0.5050

Best threshold: 0.85
Best accuracy : 0.6366666666666667
